# Keras Tuner

In [1]:
# This code Limits the gpu usage percentage 
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.6
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import kerastuner as kt

In [3]:
print(tf.__version__)

2.1.0


In [4]:
df = pd.read_csv('../AirQualityIndex-Keras-Tuner/Real_Combine.csv')
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [5]:
X = df.iloc[:,:-1] # Independent features 
y = df.iloc[:,-1] # Dependent features

In [6]:
X.shape

(1093, 8)

In [7]:
X.head()

,T,TM,Tm,SLP,H,VV,V,VM
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2


In [8]:
y.head()

0    219.720833
1    182.187500
2    154.037500
3    223.208333
4    200.645833
Name: PM 2.5, dtype: float64

In [9]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense

def model_builder(hp):
    
    model = keras.Sequential()
    
    hp_units = hp.Int('units',min_value=32,max_value=512,step=32)
    model.add(keras.layers.Dense(units=hp_units,activation='relu'))
    model.add(keras.layers.Dense(10))
    
    hp_learning_rate = hp.Choice('learning_rate',values = [1e-2,1e-3,1e-4])
    model.compile(optmizer=keras.optimizers.Adam(learning_rate = hp_learning_rate),
                                                loss='mean_absolute_error',
                                                metrics=['val_mean_absolute_error'])
    return model
    

In [10]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

### Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available 
- RandomSearch
- Hyperband
- BayesianOptimization
- Sklearn


hypermodel,
    objective,
    max_epochs,
    factor=3,
    hyperband_iterations=1,
    seed=None,
    hyperparameters=None,
    tune_new_entries=True,
    allow_new_entries=True

tuner = kt.Hyperband(model_builder,
                     objective='val_mean_absolute_error',
                    max_epochs=10,
                    factor=3,
                    directory='my_dir')

In [11]:
tuner = kt.RandomSearch(
    build_model,
    objective='mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')

In [12]:
tuner.search_space_summary()

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [14]:
tuner.search(X_train.values, y_train.values,
             epochs=5,
             validation_data=(X_test.values, y_test.values))

Train on 765 samples, validate on 328 samples
Epoch 1/5
765/765 [==============================] - ETA: 18s - loss: 149.2592 - mean_absolute_error: 149.25 - ETA: 0s - loss: nan - mean_absolute_error: nan           - 1s 1ms/sample - loss: nan - mean_absolute_error: nan - val_loss: 66.3208 - val_mean_absolute_error: 66.3208
Epoch 2/5
765/765 [==============================] - ETA: 0s - loss: 60.5695 - mean_absolute_error: 60.569 - ETA: 0s - loss: 67.6063 - mean_absolute_error: 67.606 - 0s 156us/sample - loss: nan - mean_absolute_error: nan - val_loss: 66.6536 - val_mean_absolute_error: 66.6536
Epoch 3/5
765/765 [==============================] - ETA: 0s - loss: 73.3082 - mean_absolute_error: 73.308 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 153us/sample - loss: nan - mean_absolute_error: nan - val_loss: 65.2831 - val_mean_absolute_error: 65.2831
Epoch 4/5
765/765 [==============================] - ETA: 0s - loss: 55.8352 - mean_absolute_error: 55.835 - ETA: 0s - loss: n

d:\anacondafiles\envs\deeplearning\lib\site-packages\kerastuner\engine\metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Train on 765 samples, validate on 328 samples
Epoch 1/5
765/765 [==============================] - ETA: 8s - loss: 121.0531 - mean_absolute_error: 121.053 - ETA: 0s - loss: nan - mean_absolute_error: nan          - 1s 684us/sample - loss: nan - mean_absolute_error: nan - val_loss: 65.6695 - val_mean_absolute_error: 65.6695
Epoch 2/5
765/765 [==============================] - ETA: 0s - loss: 62.7108 - mean_absolute_error: 62.710 - ETA: 0s - loss: 68.2081 - mean_absolute_error: 68.208 - 0s 159us/sample - loss: nan - mean_absolute_error: nan - val_loss: 61.2076 - val_mean_absolute_error: 61.2076
Epoch 3/5
765/765 [==============================] - ETA: 0s - loss: 59.6114 - mean_absolute_error: 59.611 - ETA: 0s - loss: nan - mean_absolute_error: nan        - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 203us/sample - loss: nan - mean_absolute_error: nan - val_loss: 66.5109 - val_mean_absolute_error: 66.5109
Epoch 4/5
765/765 [==============================] - ETA: 0s - loss: 65.2096 

Train on 765 samples, validate on 328 samples
Epoch 1/5
765/765 [==============================] - ETA: 13s - loss: 130.0913 - mean_absolute_error: 130.09 - ETA: 0s - loss: 90.8225 - mean_absolute_error: 90.8225   - 1s 1ms/sample - loss: nan - mean_absolute_error: nan - val_loss: 74.2944 - val_mean_absolute_error: 74.2944
Epoch 2/5
765/765 [==============================] - ETA: 0s - loss: 79.3463 - mean_absolute_error: 79.346 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 155us/sample - loss: nan - mean_absolute_error: nan - val_loss: 63.4380 - val_mean_absolute_error: 63.4379
Epoch 3/5
765/765 [==============================] - ETA: 0s - loss: 67.9843 - mean_absolute_error: 67.984 - ETA: 0s - loss: 67.9941 - mean_absolute_error: 67.994 - 0s 148us/sample - loss: nan - mean_absolute_error: nan - val_loss: 64.2715 - val_mean_absolute_error: 64.2715
Epoch 4/5
765/765 [==============================] - ETA: 0s - loss: 57.5773 - mean_absolute_error: 57.577 - ETA: 0s - loss: n

Train on 765 samples, validate on 328 samples
Epoch 1/5
765/765 [==============================] - ETA: 15s - loss: 104.5434 - mean_absolute_error: 104.54 - ETA: 0s - loss: 109.9882 - mean_absolute_error: 109.9882 - ETA: 0s - loss: 108.6768 - mean_absolute_error: 108.676 - 1s 1ms/sample - loss: nan - mean_absolute_error: nan - val_loss: 87.6563 - val_mean_absolute_error: 87.6563
Epoch 2/5
765/765 [==============================] - ETA: 0s - loss: 88.7476 - mean_absolute_error: 88.747 - ETA: 0s - loss: 70.6400 - mean_absolute_error: 70.640 - 0s 184us/sample - loss: nan - mean_absolute_error: nan - val_loss: 63.6789 - val_mean_absolute_error: 63.6789
Epoch 3/5
765/765 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 179us/sample - loss: nan - mean_absolute_error: nan - val_loss: 64.6172 - val_mean_absolute_error: 64.6172
Epoch 4/5
765/765 [==============================] - ETA: 0s - loss: 92.8865 - mean_

Train on 765 samples, validate on 328 samples
Epoch 1/5
765/765 [==============================] - ETA: 13s - loss: 119.9084 - mean_absolute_error: 119.90 - ETA: 0s - loss: nan - mean_absolute_error: nan           - 1s 1ms/sample - loss: nan - mean_absolute_error: nan - val_loss: 66.0547 - val_mean_absolute_error: 66.0547
Epoch 2/5
765/765 [==============================] - ETA: 0s - loss: 69.5797 - mean_absolute_error: 69.579 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 175us/sample - loss: nan - mean_absolute_error: nan - val_loss: 63.6837 - val_mean_absolute_error: 63.6837
Epoch 3/5
765/765 [==============================] - ETA: 0s - loss: 61.0990 - mean_absolute_error: 61.099 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 157us/sample - loss: nan - mean_absolute_error: nan - val_loss: 63.5309 - val_mean_absolute_error: 63.5308
Epoch 4/5
765/765 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mea

INFO:tensorflow:Oracle triggered exit


In [23]:
tuner.results_summary()

In [71]:
tuner.search(X_train.values, y_train.values,
             epochs=5,
            validation_data=X_test.values,y_test.values)

Train on 765 samples
Epoch 1/7
765/765 [==============================] - ETA: 21s - loss: 103.0716 - mean_absolute_error: 103.07 - ETA: 3s - loss: 99.9834 - mean_absolute_error: 99.9834   - ETA: 1s - loss: 86.9621 - mean_absolute_error: 86.962 - ETA: 0s - loss: 83.4764 - mean_absolute_error: 83.476 - ETA: 0s - loss: 79.4234 - mean_absolute_error: 79.423 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 1s 2ms/sample - loss: nan - mean_absolute_error: nan
Epoch 2/7
765/765 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 267us/sample - loss: nan - mean_absolute_error: nan
Epoch 3/7
765/765 [==============================] - ETA: 0s - loss: 75.7126 - mean_absolute_error: 75.712 - ETA: 0s - loss: 62.3950 - mean_absolute_error: 62.395 - ETA: 0s - loss: nan - mean_absolute_error: nan        - ETA: 0s - l

d:\anacondafiles\envs\deeplearning\lib\site-packages\kerastuner\engine\metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Train on 765 samples
Epoch 1/7
765/765 [==============================] - ETA: 15s - loss: 112.3372 - mean_absolute_error: 112.33 - ETA: 0s - loss: nan - mean_absolute_error: nan           - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - 1s 1ms/sample - loss: nan - mean_absolute_error: nan
Epoch 2/7
765/765 [==============================] - ETA: 0s - loss: 71.6040 - mean_absolute_error: 71.604 - ETA: 0s - loss: 67.3798 - mean_absolute_error: 67.379 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 173us/sample - loss: nan - mean_absolute_error: nan
Epoch 3/7
765/765 [==============================] - ETA: 0s - loss: 65.3010 - mean_absolute_error: 65.301 - ETA: 0s - loss: nan - mean_absolute_error: nan        - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 176us/sample - loss: nan - mean_absolute_error: nan
Epoch 4/7
765/765 [==============================] - ETA: 0s - loss: 43.7711 - mean_absolute_error: 43.771 - ETA: 0s

Train on 765 samples
Epoch 1/7
765/765 [==============================] - ETA: 20s - loss: 88.6679 - mean_absolute_error: 88.66 - ETA: 1s - loss: 432.6472 - mean_absolute_error: 432.647 - ETA: 0s - loss: nan - mean_absolute_error: nan          - 1s 1ms/sample - loss: nan - mean_absolute_error: nan
Epoch 2/7
765/765 [==============================] - ETA: 0s - loss: 54.7780 - mean_absolute_error: 54.778 - ETA: 0s - loss: 72.1932 - mean_absolute_error: 72.193 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 193us/sample - loss: nan - mean_absolute_error: nan
Epoch 3/7
765/765 [==============================] - ETA: 0s - loss: 74.6917 - mean_absolute_error: 74.691 - ETA: 0s - loss: 60.7344 - mean_absolute_error: 60.734 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 190us/sample - loss: nan - mean_absolute_error: nan
Epoch 4/7
765/765 [==============================] - ETA: 0s - loss: 52.1751 - mean_absolute_error: 52.175 - ETA: 0s - loss: 55.4984 - mean_absolute_

Train on 765 samples
Epoch 1/7
765/765 [==============================] - ETA: 12s - loss: 107.0630 - mean_absolute_error: 107.06 - ETA: 0s - loss: 81.3953 - mean_absolute_error: 81.3953   - ETA: 0s - loss: nan - mean_absolute_error: nan        - 1s 885us/sample - loss: nan - mean_absolute_error: nan
Epoch 2/7
765/765 [==============================] - ETA: 0s - loss: 69.5325 - mean_absolute_error: 69.532 - ETA: 0s - loss: 66.4776 - mean_absolute_error: 66.477 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 199us/sample - loss: nan - mean_absolute_error: nan
Epoch 3/7
765/765 [==============================] - ETA: 0s - loss: 75.8975 - mean_absolute_error: 75.897 - ETA: 0s - loss: nan - mean_absolute_error: nan        - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 241us/sample - loss: nan - mean_absolute_error: nan
Epoch 4/7
765/765 [==============================] - ETA: 0s - loss: 82.2518 - mean_absolute_error: 82.251

Train on 765 samples
Epoch 1/7
765/765 [==============================] - ETA: 20s - loss: 93.9442 - mean_absolute_error: 93.94 - ETA: 1s - loss: 183.8867 - mean_absolute_error: 183.886 - ETA: 0s - loss: nan - mean_absolute_error: nan          - ETA: 0s - loss: nan - mean_absolute_error: na - 1s 1ms/sample - loss: nan - mean_absolute_error: nan
Epoch 2/7
765/765 [==============================] - ETA: 0s - loss: 65.1980 - mean_absolute_error: 65.198 - ETA: 0s - loss: nan - mean_absolute_error: nan        - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 209us/sample - loss: nan - mean_absolute_error: nan
Epoch 3/7
765/765 [==============================] - ETA: 0s - loss: 62.1478 - mean_absolute_error: 62.147 - ETA: 0s - loss: 69.8938 - mean_absolute_error: 69.893 - ETA: 0s - loss: 63.8551 - mean_absolute_error: 63.855 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 253us/sample - loss: nan - mean_absolute_error: nan
Epoch 4/7
765/765 [==============================] - 

INFO:tensorflow:Oracle triggered exit
